In [1]:
import datetime

In [2]:
from typing import List

In [3]:
import requests

In [5]:
import pandas as pd

In [12]:
pip install tableauhyperapi

Note: you may need to restart the kernel to use updated packages.


In [14]:
from tableauhyperapi import (
    HyperProcess,
    Connection,
    TableDefinition,
    SqlType,
    Telemetry,
    Inserter,
    CreateMode,
    TableName
)
from tableauhyperapi import escape_string_literal

In [18]:
pip install tableau_api_lib

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.3.0
    Uninstalling typing-extensions-4.3.0:
      Successfully uninstalled typing-extensions-4.3.0
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.3.0
    Uninstalling importlib-metadata-3.3.0:
      Successfully uninstalled importlib-metadata-3.3.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
wasabi 1.1.1 requires typing-extensions<4.5.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.5.0 which is incompatible.
thinc 8.1.9 requires typing-extensions<4.5.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.5.0 which is incompatible.
tensorflow 2.4.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.5.0 which is incompatible.
spacy 3.5.1 requires typing-extensions<4.5.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.5.0 which is incompatible.
confection 0.0.4 requires typing-extensions<4.5.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.5.0 which is incom

In [19]:
#from tableau_api_lib import TableauServerConnection # publish the hyperfile

In [20]:
# basic get request against Gemini's REST API: get all trading symbols (currency pairs)

In [44]:
response = requests.get(url= "https://api.gemini.com/v1/symbols")

In [45]:
type(response)

requests.models.Response

In [46]:
# Data we care about is packed inside the response's JSON component

In [47]:
all_symbols = response.json()

In [48]:
# we can peek at some of the symbols the Gemini Exchange returned to us. 

In [49]:
all_symbols[:10]

['1inchusd',
 'aaveusd',
 'alcxusd',
 'aliusd',
 'ampusd',
 'ankrusd',
 'apeusd',
 'api3usd',
 'atomusd',
 'audiousd']

In [50]:
# get the raw response JSON data into a more analysis-friendly format

In [51]:
def get_candles_df(json_data:str)->pd.DataFrame:
    "Converts raw JSON pricing data into Pandas dataframe"
    candles_df = pd.DataFrame(json_data)
    candles_df.columns = ["time", "open_price", "high_price", "low_price", "close_price", "volume"]
    candles_df["time"] = candles_df["time"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
    return candles_df

In [55]:
def get_crypto_prices_df(symbols:List[str])->pd.DataFrame:
    """Queries pricing data from Gemini for each of available crypto symbols"""
    base_url = "https://api.gemini.com"
    api_version = "v2"
    time_frame="1day" 
    crypto_prices_df = pd.DataFrame() # empty df to be filled
    
    for symbol in symbols:
        print(f"Fetching pricing data for crypto symbol '{symbol}'...")
        url = f"{base_url}/{api_version}/candles/{symbol}/{time_frame}"
        response = requests.get(url=url)
        candles_df = get_candles_df(json_data=response.json())
        candles_df["symbol"] = symbol
        crypto_prices_df=crypto_prices_df.append(candles_df)
    return crypto_prices_df

In [56]:
crypto_prices_df = get_crypto_prices_df(symbols = all_symbols)

Fetching pricing data for crypto symbol '1inchusd'...


TypeError: get_candles_df() got an unexpected keyword argument 'index_col'